In [14]:
import pandas as pd
import datetime
import numpy as np
import pickle
from scipy import spatial
import itertools
from scipy import signal
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import urllib2 
import urllib
import operator
import string
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.embed import components 
from bokeh.io import hplot

In [15]:
#READ IN THE YELP FILES to a pandas data frame
user = pd.read_csv("yelp_academic_dataset_user.csv", index_col = 'user_id')
print "user read-in complete"
print(user.shape)

business = pd.read_csv("yelp_academic_dataset_business.csv", na_filter=False, index_col = 'business_id')
print "business read-in complete"
print(business.shape)

reviews = pd.read_csv("yelp_academic_dataset_review.csv", index_col = 'review_id')
print "reviews read-in complete"
print(reviews.shape)

user read-in complete
(552339, 22)
business read-in complete
(77445, 97)
reviews read-in complete
(2225213, 9)


In [16]:
transformer = TfidfTransformer() #to do tfidf weighting on all of the bag_of_words vectors
nreturn = 10
nplot = 5

#reviews = pd.read_pickle('reviews_random')
reviews = pd.read_pickle('rev_and_business')
print reviews[0:5]

              business_id       date  day  month  stars  \
0  5UmKMjUEUNdYWqANhGckJw 2012-08-01    1      8      4   
1  5UmKMjUEUNdYWqANhGckJw 2014-02-13   13      2      5   
2  5UmKMjUEUNdYWqANhGckJw 2015-10-31   31     10      5   
3  WaHXyBwljbKNPmmJZn5j8Q 2012-08-01    1      8      3   
4  WaHXyBwljbKNPmmJZn5j8Q 2014-07-26   26      7      3   

                                          text_clean  year business_name  \
0  mr hoagie institution walking seem like throwb...  2012     Mr Hoagie   
1  excellent food superb customer service miss ma...  2014     Mr Hoagie   
2  yes place little dated opened weekend staff al...  2015     Mr Hoagie   
3                                          shop save  2012  Shop 'n Save   
4  living alone love deals meat also love faygo p...  2014  Shop 'n Save   

   business_stars          city state  
0             4.5    Dravosburg    PA  
1             4.5    Dravosburg    PA  
2             4.5    Dravosburg    PA  
3             3.0  West Miff

In [6]:
my_city = 'Las Vegas'
#my_city = 'Los Angeles'

reviews_total = []
reviews_subset = reviews[reviews['city'] == my_city]

begin_date = min(reviews_subset['date'])
end_date = max(reviews_subset['date'])

datelist = np.arange(begin_date, '2020-01-01', dtype='datetime64[D]')

next_date = begin_date

i=1
while next_date < end_date:
    next_date = next_date + pd.DateOffset(months=+1, days=0)
    datelist[i] = next_date
    i = i+1

datelist = datelist[0:i]
    
dlen = len(datelist)

print len(reviews_subset)
for i in range(0,dlen-1):
    this_month = reviews_subset[(reviews_subset['date'] > datelist[i]) & (reviews_subset['date'] <= datelist[i+1])]
    #if len(this_month) > 0:
    rtext = this_month['text_clean']
    if len(rtext) > 100:
        rtext = rtext[0:100]
    rtext_combined = string.join(rtext)
    reviews_total.append(rtext_combined)

print len(datelist)
print len(reviews_total)

861536
136
135


In [19]:
reviews_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 400, ngram_range=(1,1)) 
reviews_features = reviews_vec.fit_transform(reviews_total)
reviews_tfidf = transformer.fit_transform(reviews_features)
reviews_features = reviews_features.toarray()
reviews_tfidf = reviews_tfidf.toarray()


reviews2_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 400, ngram_range=(2,2)) 
reviews2_features = reviews2_vec.fit_transform(reviews_total)
reviews2_tfidf = transformer.fit_transform(reviews2_features)
reviews2_features = reviews2_features.toarray()
reviews2_tfidf = reviews2_tfidf.toarray()


reviews3_vec = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 400, ngram_range=(3,3)) 
reviews3_features = reviews3_vec.fit_transform(reviews_total)
reviews3_tfidf = transformer.fit_transform(reviews3_features)
reviews3_features = reviews3_features.toarray()
reviews3_tfidf = reviews3_tfidf.toarray()

vocab = reviews_vec.get_feature_names()
vocab2 = reviews2_vec.get_feature_names()
vocab3 = reviews3_vec.get_feature_names()

In [20]:
print vocab
print vocab2
print vocab3

[u'able', u'absolutely', u'across', u'actually', u'almost', u'also', u'although', u'always', u'amazing', u'another', u'anything', u'area', u'around', u'arrived', u'ask', u'asked', u'ass', u'atmosphere', u'away', u'awesome', u'back', u'bad', u'bar', u'bathroom', u'bathrooms', u'beautiful', u'bed', u'beds', u'bellagio', u'best', u'better', u'big', u'bit', u'book', u'booked', u'breakfast', u'bring', u'buffet', u'caesar', u'caesars', u'cafe', u'call', u'called', u'came', u'card', u'care', u'casino', u'casinos', u'center', u'charge', u'cheap', u'check', u'checked', u'circus', u'clean', u'close', u'club', u'coffee', u'come', u'comfortable', u'coming', u'cool', u'could', u'couple', u'course', u'crowd', u'crowded', u'customer', u'dance', u'day', u'days', u'deal', u'decent', u'decided', u'decor', u'definitely', u'delicious', u'desk', u'different', u'dining', u'dinner', u'disappointed', u'done', u'door', u'double', u'drink', u'drinks', u'early', u'easy', u'eat', u'either', u'elevators', u'else',

In [8]:
i=1
dist = reviews_tfidf[i] #one row of word features
count_dict = dict(zip(vocab, dist))
sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
word, freq = zip(*sorted_count_dict)

dist2 = reviews2_tfidf[i] #one row of word features
count_dict2 = dict(zip(vocab2, dist2))
sorted_count_dict2 = sorted(count_dict2.items(), key=operator.itemgetter(1),reverse=True)
word2, freq2 = zip(*sorted_count_dict2)

dist3 = reviews3_tfidf[i] #one row of word features
count_dict3 = dict(zip(vocab3, dist3))
sorted_count_dict3 = sorted(count_dict3.items(), key=operator.itemgetter(1),reverse=True)
word3, freq3 = zip(*sorted_count_dict3)

ugram = word[0:nreturn]
bgram = word2[0:nreturn]
tgram = word3[0:nreturn]

result = pd.DataFrame({'unigram' : ugram, 'bigram' : bgram, 'trigram' : tgram})
print result

            bigram            trigram unigram
0  everything else    hotel las vegas   since
1        good food  room smelled like    wynn
2      many people    room nice clean    show
3        room nice   non smoking room   money
4     one favorite   last time stayed   cheap
5        view room    one best hotels   years
6    circus circus      stay new york  course
7      much better    first time ever   rooms
8        one thing  front desk person      go
9     mandalay bay    best hotel ever   still


In [9]:
word1_list = []
freq1 = []

word2_list = []
freq2 = []

word3_list = []
freq3 = []

for i in range(dlen-1):
    dist = reviews3_tfidf[i] #one row of word features
    count_dict = dict(zip(vocab3, dist))
    sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
    word, freq = zip(*sorted_count_dict)
    
    #print datelist[i], map(lambda x: x.encode('ascii'), word[0:1]), freq[0]
    word3_list.append(word[0:1])
    freq3.append(freq[0])
    
    dist = reviews2_tfidf[i] #one row of word features
    count_dict = dict(zip(vocab2, dist))
    sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
    word, freq = zip(*sorted_count_dict)
    
    #print datelist[i], map(lambda x: x.encode('ascii'), word[0:1]), freq[0]
    word2_list.append(word[0:1])
    freq2.append(freq[0])
    
    dist = reviews_tfidf[i] #one row of word features
    count_dict = dict(zip(vocab, dist))
    sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1),reverse=True)
    word, freq = zip(*sorted_count_dict)
    
    #print datelist[i], map(lambda x: x.encode('ascii'), word[0:1]), freq[0]
    word1_list.append(word[0:1])
    freq1.append(freq[0])    

word_tot = []    
for w in word1_list:
    word_tot.append(w)
for w in word2_list:
    word_tot.append(w)
for w in word3_list:
    word_tot.append(w)


freq_tot = []   
color_list_long = []
for f in freq1:
    freq_tot.append(f)
    color_list_long.append('blue')
for f in freq2:
    freq_tot.append(f)
    color_list_long.append('red')
for f in freq3:
    freq_tot.append(f)   
    color_list_long.append('green')
    
    
datelist_triple = []    
for i in range(3):
    for d in datelist[0:len(freq1)]:
        datelist_triple.append(d)
        
print datelist[-3]        

2015-11-18


In [10]:
#output_notebook()

In [11]:
print len(word_tot)
print len(datelist_triple)
print len(freq_tot)
print len(color_list_long)

from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, HoverTool

TOOLS = "pan,wheel_zoom,box_zoom,reset"
color_list = ['blue','red', 'green', 'orange', 'black']

source = ColumnDataSource(data = dict(x=datelist_triple, y=freq_tot, words=word_tot, the_colors=color_list_long))

hover = HoverTool(tooltips=  """
                 <div>
                 <span style="font-size: 15px; font-weight: bold;">@words</span>
                 </div>
                  """
                 )

plot1 = figure(tools=["pan","wheel_zoom","box_zoom","reset",hover], title='Frequently used words and phrases', x_axis_label='date', 
               x_axis_type='datetime', y_axis_label = 'relative word frequency')


plot1.square('x', 'y', source=source, color=color_list_long)


plot1.xaxis.axis_label_text_font_size = "20pt"
plot1.yaxis.axis_label_text_font_size = "20pt"
plot1.xaxis.axis_label_text_font_style = "normal"
plot1.yaxis.axis_label_text_font_style = "normal"
plot1.yaxis.major_label_text_font_size = "15pt"
plot1.xaxis.major_label_text_font_size = "15pt"

label1 = Label(x=400, y=470, x_units='screen', y_units='screen', text='blue -- unigrams', render_mode='css',
      border_line_color='white', background_fill_color='white')
label2 = Label(x=400, y=500, x_units='screen', y_units='screen', text='red -- biigrams', render_mode='css',
      border_line_color='white', background_fill_color='white')
label3 = Label(x=400, y=530, x_units='screen', y_units='screen', text='green -- trigrams', render_mode='css',
      border_line_color='white', background_fill_color='white')

plot1.add_layout(label1)
plot1.add_layout(label2)
plot1.add_layout(label3)

p = plot1

output_file("result.html")
#script, div = components(p)
# show the results
show(p)

405
405
405
405


In [23]:
city_list = reviews['city'].unique()
    
rev_by_city = reviews.groupby(['city']).count()
rev_by_city2 = rev_by_city[rev_by_city['state'] > 10000]

print rev_by_city2

city_list = []

for c in rev_by_city2.index:
    city_list.append(c)

for c in city_list:
    print c,',',reviews[reviews['city']==c].state.iloc[0]
    

#MontrMontr\xc3\xa9al is Montréal

                 business_id    date     day   month   stars  text_clean  \
city                                                                       
Champaign              11188   11188   11188   11188   11188       11188   
Chandler               60321   60321   60321   60321   60321       60321   
Charlotte             122655  122655  122655  122655  122655      122655   
Edinburgh              28196   28196   28196   28196   28196       28196   
Gilbert                43769   43769   43769   43769   43769       43769   
Glendale               35829   35829   35829   35829   35829       35829   
Goodyear               10150   10150   10150   10150   10150       10150   
Henderson              78438   78438   78438   78438   78438       78438   
Las Vegas             861536  861536  861536  861536  861536      861536   
Madison                49818   49818   49818   49818   49818       49818   
Mesa                   59723   59723   59723   59723   59723       59723   
Montréal    

In [13]:
review_rate = np.zeros(len(business_subset))
first_date = np.arange('2000-01-01', '2020-01-01', dtype='datetime64[D]')
last_date = np.arange('2000-01-01', '2020-01-01', dtype='datetime64[D]')

for i in range(len(business_subset)):
    bus_id = business_subset.index[i]
    review_subset = reviews[reviews['business_id'] == bus_id]
    first_date_here = min(review_subset['date'])
    last_date_here = max(review_subset['date'])
    delta = last_date_here - first_date_here
    if i %100 == 0:
        print "i =", i

#my_colors = list(map(calc_rate, business_subset))

NameError: name 'business_subset' is not defined